In [1]:
!pip install --upgrade --user google-cloud-aiplatform google-cloud-bigquery

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.1/239.1 kB 20.9 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
# Get project ID
PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-west1" # @param {type:"string"}
print(PROJECT_ID)

qwiklabs-gcp-01-173eaa26d171


In [2]:
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=LOCATION)

print("Initialized")

Initialized


In [3]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part

def answer_question_gemini(prompt):
  model = GenerativeModel("gemini-pro")
  response = model.generate_content(
    prompt,
    generation_config={
        "max_output_tokens": 8192,
        "temperature": 0.5,
        "top_p": 0.5,
        "top_k": 10,
    },
  stream=False,
  )
  try:
    return response.text
  except:
    print("An Error Ocuured Cleaning the Data")
    return "An Error Ocuured Cleaning the Data"

In [4]:
def run_search(question):
  from google.cloud import bigquery

  client = bigquery.Client()

  sql = """
      SELECT query.query, base.title, base.body
      FROM VECTOR_SEARCH(
      TABLE `bbc_news.bbc_news_with_embeddings_table`, 'text_embedding',
      (
      SELECT text_embedding, content AS query
      FROM ML.GENERATE_TEXT_EMBEDDING(MODEL `bbc_news.bq_embedding_model`,
          (SELECT @question AS content))),
      top_k => 5)
      """

  job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter("question", "STRING", question),
    ]
  )

  query_job = client.query(sql, job_config=job_config)

  data = ""
  for row in query_job:
    data += row.body + "\n"

  return data

In [5]:
def build_prompt(data, question):
  prompt = """
    Instructions: Answer the question using the following Context.

    Context: {0}

    Question: {1}
  """.format(data, question)
  return prompt

In [6]:
from IPython.core.display import display, HTML

def answer_question(question):

  data = run_search(question)
  display("Retrieved Data:")
  display(data)
  display(" . . . ")
  prompt = build_prompt(data, question)
  answer_gemini = answer_question_gemini(prompt)

  return answer_gemini

In [7]:
QUESTION = "Tell me about the US Economy"

answer_gemini = answer_question(QUESTION)
display("User Question:")
display(QUESTION)
display("--------------------------------")
display("Gemini Answer:")
display(answer_gemini)

'Retrieved Data:'

'Most areas of the US saw their economy continue to expand in December and early January, the US Federal Reserve said in its latest Beige Book report.\n\nOf the 12 US regions it identifies for the study, 11 showed stronger economic growth, with only the Cleveland area falling behind with a "mixed" rating. Consumer spending was higher in December than November, and festive sales were also up on 2003. The employment picture also improved, the Fed said.\n\n"Labour markets firmed in a number of districts, but wage pressures generally remained modest," the Beige Book said. "Several districts reported higher prices for building materials and manufacturing inputs, but most reported steady or only slightly higher overall price levels." The report added that residential real estate activity remained strong and that commercial real estate activity strengthened in most districts. "Office leasing was especially brisk in Washington DC, and New York City, two of the nation\'s strongest commercial ma

' . . . '

'User Question:'

'Tell me about the US Economy'

'--------------------------------'

'Gemini Answer:'

"## The US Economy in 2004: A Summary\n\nBased on the provided context, here's a summary of the US economy in 2004:\n\n**Overall:**\n\n* **Expansion:** The US economy continued to expand throughout 2004, with most regions experiencing stronger growth. \n* **Consumer Spending:** Consumer spending was a key driver of growth, particularly during the holiday season.\n* **Employment:** The employment picture improved, with job creation exceeding expectations for the year. However, wage pressures remained modest.\n* **Real Estate:** Both residential and commercial real estate activity remained strong.\n* **Trade Deficit:** The trade deficit widened to a record high, raising concerns about the impact on the dollar.\n\n**Key Developments:**\n\n* **Strong third-quarter growth:** The economy grew at an annual rate of 3.7% in the third quarter, helped by strong consumer spending.\n* **Job creation:** The US created 2.2 million jobs in 2004, the best showing in five years.\n* **Interest rate hikes